# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [41]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///../../../db/dm_eyf.db

In [2]:
dataset_path = 'C:/Users/Federico/Desktop/Maestria Data mining/DM EyF/datasets/'
dataset_file = 'competencia_01.csv'

In [3]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("../../../datasets/competencia_01.csv")

,Success


In [4]:
df=pd.read_csv(dataset_path + dataset_file)

In [5]:
campos=df.columns.to_list()
print(campos[-1])
campos.pop(-1) #elimino clase ternaria
print(campos[-1])

clase_ternaria
Visa_mpagominimo


Ahora creo una tabla con las nuevas columnas, tengo que hacer la macro por que estoy dividiendo por 0.. 

In [6]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0)

,Success


In [7]:
%%sql
CREATE OR REPLACE MACRO div_sin_cero(a, b) as
    case
        when ifnull(b, 0) = 0 THEN NULL
        else ifnull(a, 0) / ifnull(b, 1)
    end

,Success


In [8]:
# Separar las columnas por tipo
master = [col for col in campos if 'Master' in col]
visa = [col for col in campos if 'Visa' in col]

sumas = []

for m_col, v_col in zip(master, visa):
    # Crear la suma usando la función 'suma_sin_null'
    suma = f"\n    suma_sin_null({m_col}, {v_col}) as {m_col.replace('Master_', '')}_total"
    sumas.append(suma)

consulta_sql = f"""
select 
    numero_de_cliente,
    {', '.join(sumas)}
from competencia_01;
"""

print(consulta_sql)


select 
    numero_de_cliente,
    
    suma_sin_null(Master_delinquency, Visa_delinquency) as delinquency_total, 
    suma_sin_null(Master_status, Visa_status) as status_total, 
    suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as mfinanciacion_limite_total, 
    suma_sin_null(Master_Fvencimiento, Visa_Fvencimiento) as Fvencimiento_total, 
    suma_sin_null(Master_Finiciomora, Visa_Finiciomora) as Finiciomora_total, 
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as msaldototal_total, 
    suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as msaldopesos_total, 
    suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as msaldodolares_total, 
    suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as mconsumospesos_total, 
    suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as mconsumosdolares_total, 
    suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as mlimitecompra_total, 
    suma_sin_null(Master_madelantopesos, Vi

In [9]:
%%sql
create or replace table competencia_01_suma_division as
select
    numero_de_cliente,
    foto_mes,
    suma_sin_null(Master_delinquency, Visa_delinquency) as delinquency_total,
    suma_sin_null(Master_status, Visa_status) as status_total,
    suma_sin_null(Master_mfinanciacion_limite, Visa_mfinanciacion_limite) as mfinanciacion_limite_total,
    suma_sin_null(Master_msaldototal, Visa_msaldototal) as msaldototal_total,
    suma_sin_null(Master_msaldopesos, Visa_msaldopesos) as msaldopesos_total,
    suma_sin_null(Master_msaldodolares, Visa_msaldodolares) as msaldodolares_total,
    suma_sin_null(Master_mconsumospesos, Visa_mconsumospesos) as mconsumospesos_total,
    suma_sin_null(Master_mconsumosdolares, Visa_mconsumosdolares) as mconsumosdolares_total,
    suma_sin_null(Master_mlimitecompra, Visa_mlimitecompra) as mlimitecompra_total,
    suma_sin_null(Master_madelantopesos, Visa_madelantopesos) as madelantopesos_total,
    suma_sin_null(Master_madelantodolares, Visa_madelantodolares) as madelantodolares_total,
    suma_sin_null(Master_mpagado, Visa_mpagado) as mpagado_total,
    suma_sin_null(Master_mpagospesos, Visa_mpagospesos) as mpagospesos_total,
    suma_sin_null(Master_mpagosdolares, Visa_mpagosdolares) as mpagosdolares_total,
    suma_sin_null(Master_mconsumototal, Visa_mconsumototal) as mconsumototal_total,
    suma_sin_null(Master_cconsumos, Visa_cconsumos) as cconsumos_total,
    suma_sin_null(Master_cadelantosefectivo, Visa_cadelantosefectivo) as cadelantosefectivo_total,
    suma_sin_null(Master_mpagominimo, Visa_mpagominimo) as mpagominimo_total,

    suma_sin_null(ctarjeta_visa, ctarjeta_master) as cant_tarjetas,
    suma_sin_null(ctarjeta_visa_transacciones, ctarjeta_master_transacciones) as cant_transac_tarjetas,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as mtarjetas_consumo,
    suma_sin_null(cprestamos_personales,cprestamos_prendarios) as c_prestamos,
    suma_sin_null(mpayroll, mpayroll2) as m_payroll_total, -- monto acreditacion
    suma_sin_null(cpayroll_trx, cpayroll2_trx) as m_cpayroll_total, -- cantidad de acreditaciones
    suma_sin_null(minversion1_dolares, minversion1_pesos) as minversion1_total,
    suma_sin_null(mplazo_fijo_dolares, mplazo_fijo_pesos) as mplazofijo_total,
    suma_sin_null(suma_sin_null(cseguro_vida, cseguro_auto), suma_sin_null(cseguro_vivienda, cseguro_accidentes_personales)) as cseguros_total,

    div_sin_cero(mconsumototal_total, mlimitecompra_total) as ratio_uso_credito,
    div_sin_cero(mpagado_total, mconsumototal_total) as ratio_pago_vs_consumo,
    div_sin_cero(mpagominimo_total, mpagado_total) as ratio_pago_min_vs_total,
    div_sin_cero(mconsumospesos_total, mconsumosdolares_total) as ratio_consumo_pesos_dolares,
    div_sin_cero(madelantopesos_total + madelantodolares_total, mlimitecompra_total) as ratio_adelantos_vs_limite,
    div_sin_cero(cseguros_total, mconsumototal_total) as ratio_seguros_vs_consumo,

    greatest(Master_Finiciomora,Visa_Finiciomora) as fechamora_mayor,
    least(Master_Finiciomora,Visa_Finiciomora) as fechamora_menor,
    greatest(Master_fultimo_cierre,Visa_fultimo_cierre) as fechacierre_mayor,
    least(Master_fultimo_cierre,Visa_fultimo_cierre) as fechacierre_menor,
    greatest(Master_Fvencimiento, Visa_Fvencimiento) as fvencimiento_mayor,
    least(Master_Fvencimiento, Visa_Fvencimiento) as fvencimiento_menor,
    greatest(Master_fechaalta, Visa_fechaalta) as fechaalta_mayor,
    least(Master_fechaalta, Visa_fechaalta) as fechalta_menor,

    ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6
from competencia_01


,Success


In [49]:
%%sql
select 
    *
from competencia_01_suma_division


,numero_de_cliente,foto_mes,delinquency_total,status_total,mfinanciacion_limite_total,msaldototal_total,msaldopesos_total,msaldodolares_total,mconsumospesos_total,mconsumosdolares_total,...,ratio_seguros_vs_consumo,fechamora_mayor,fechamora_menor,fechacierre_mayor,fechacierre_menor,fvencimiento_mayor,fvencimiento_menor,fechaalta_mayor,fechalta_menor,cliente_antiguedad_6
0,444498051,202106,0,0,1252889.55,113591.11,126043.08,0.0,48766.81,26.16,...,0.000041,NaN,NaN,0.0,0.0,-334.0,-699.0,2590.0,2590.0,9
1,445447925,202106,0,0,582746.40,20451.91,23079.93,0.0,5793.15,0.00,...,0.000000,NaN,NaN,7.0,0.0,-852.0,-1644.0,5750.0,4222.0,9
2,441515267,202106,0,0,1061485.22,33689.48,39517.76,0.0,6748.60,0.00,...,0.000000,NaN,NaN,0.0,0.0,-638.0,-1004.0,4507.0,4507.0,9
3,445241400,202106,0,0,2123054.19,79584.85,83881.25,0.0,38525.01,0.00,...,0.000000,NaN,NaN,21.0,7.0,-334.0,-548.0,5882.0,5678.0,9
4,444486607,202106,0,0,266036.40,31044.06,34472.49,0.0,10376.07,0.00,...,0.000000,NaN,NaN,0.0,0.0,-517.0,-2160.0,5518.0,3880.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1235764794,202103,0,0,304041.60,10979.86,12879.37,0.0,9110.10,0.00,...,0.000000,NaN,NaN,1.0,1.0,-1582.0,-1947.0,248.0,248.0,2
981942,1238303751,202103,0,0,810777.60,8877.85,10078.09,0.0,3703.29,0.00,...,0.000000,NaN,NaN,7.0,7.0,-1490.0,-1855.0,1442.0,1442.0,2
981943,439254013,202103,0,0,228031.20,47323.70,55078.20,0.0,42766.36,0.00,...,0.000000,NaN,NaN,1.0,1.0,-1490.0,-1855.0,1456.0,1456.0,2
981944,476440909,202103,0,0,1536365.37,45796.26,48448.24,328.0,34693.66,3.50,...,0.000000,NaN,NaN,1.0,1.0,-1855.0,-2039.0,1458.0,1256.0,2


## lag, valor del mes anterior 
## delta diferencia entre meses

In [43]:
campos = ["ctrx_quarter","cpayroll_trx","mpayroll","mpayroll2","cpayroll2_trx","mpasivos_margen",'mprestamos_personales','mcuentas_saldo','mcaja_ahorro','mcomisiones_mantenimiento',
'cproductos','cdescubierto_preacordado',"mtarjeta_visa_consumo","cprestamos_personales", "mcuenta_corriente"
]

In [44]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n    , lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}"
  nuevos_features += f"\n    , {campo} - lag_1_{campo} as delta_1_{campo}"
  nuevos_features += f"\n    , lag({campo}, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_{campo}"
  nuevos_features += f"\n    , {campo} - lag_2_{campo} as delta_2_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_lag_delta AS
select
    {nuevos_features.strip()}
FROM competencia_01
"""


In [45]:
print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_lag_delta AS
select
    , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
    , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
    , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
    , ctrx_quarter - lag_2_ctrx_quarter as delta_2_ctrx_quarter
    , lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
    , cpayroll_trx - lag_1_cpayroll_trx as delta_1_cpayroll_trx
    , lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx
    , cpayroll_trx - lag_2_cpayroll_trx as delta_2_cpayroll_trx
    , lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
    , mpayroll - lag_1_mpayroll as delta_1_mpayroll
    , lag(mpayroll, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll
    , mpayroll - l

In [50]:
%%sql
CREATE OR REPLACE TABLE competencia_01_lag_delta AS
select
    numero_de_cliente
    , foto_mes
    , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
    , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
    , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
    , ctrx_quarter - lag_2_ctrx_quarter as delta_2_ctrx_quarter
    , lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
    , cpayroll_trx - lag_1_cpayroll_trx as delta_1_cpayroll_trx
    , lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx
    , cpayroll_trx - lag_2_cpayroll_trx as delta_2_cpayroll_trx
    , lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
    , mpayroll - lag_1_mpayroll as delta_1_mpayroll
    , lag(mpayroll, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll
    , mpayroll - lag_2_mpayroll as delta_2_mpayroll
    , lag(mpayroll2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll2
    , mpayroll2 - lag_1_mpayroll2 as delta_1_mpayroll2
    , lag(mpayroll2, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll2
    , mpayroll2 - lag_2_mpayroll2 as delta_2_mpayroll2
    , lag(cpayroll2_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll2_trx
    , cpayroll2_trx - lag_1_cpayroll2_trx as delta_1_cpayroll2_trx
    , lag(cpayroll2_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll2_trx
    , cpayroll2_trx - lag_2_cpayroll2_trx as delta_2_cpayroll2_trx
    , lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen
    , mpasivos_margen - lag_1_mpasivos_margen as delta_1_mpasivos_margen
    , lag(mpasivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpasivos_margen
    , mpasivos_margen - lag_2_mpasivos_margen as delta_2_mpasivos_margen
    , lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales
    , mprestamos_personales - lag_1_mprestamos_personales as delta_1_mprestamos_personales
    , lag(mprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_personales
    , mprestamos_personales - lag_2_mprestamos_personales as delta_2_mprestamos_personales
    , lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo
    , mcuentas_saldo - lag_1_mcuentas_saldo as delta_1_mcuentas_saldo
    , lag(mcuentas_saldo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuentas_saldo
    , mcuentas_saldo - lag_2_mcuentas_saldo as delta_2_mcuentas_saldo
    , lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro
    , mcaja_ahorro - lag_1_mcaja_ahorro as delta_1_mcaja_ahorro
    , lag(mcaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro
    , mcaja_ahorro - lag_2_mcaja_ahorro as delta_2_mcaja_ahorro
    , lag(mcomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - lag_1_mcomisiones_mantenimiento as delta_1_mcomisiones_mantenimiento
    , lag(mcomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones_mantenimiento
    , mcomisiones_mantenimiento - lag_2_mcomisiones_mantenimiento as delta_2_mcomisiones_mantenimiento
    , lag(cproductos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cproductos
    , cproductos - lag_1_cproductos as delta_1_cproductos
    , lag(cproductos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cproductos
    , cproductos - lag_2_cproductos as delta_2_cproductos
    , lag(cdescubierto_preacordado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cdescubierto_preacordado
    , cdescubierto_preacordado - lag_1_cdescubierto_preacordado as delta_1_cdescubierto_preacordado
    , lag(cdescubierto_preacordado, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cdescubierto_preacordado
    , cdescubierto_preacordado - lag_2_cdescubierto_preacordado as delta_2_cdescubierto_preacordado
    , lag(mtarjeta_visa_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo
    , mtarjeta_visa_consumo - lag_1_mtarjeta_visa_consumo as delta_1_mtarjeta_visa_consumo
    , lag(mtarjeta_visa_consumo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_visa_consumo
    , mtarjeta_visa_consumo - lag_2_mtarjeta_visa_consumo as delta_2_mtarjeta_visa_consumo
    , lag(cprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_personales
    , cprestamos_personales - lag_1_cprestamos_personales as delta_1_cprestamos_personales
    , lag(cprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cprestamos_personales
    , cprestamos_personales - lag_2_cprestamos_personales as delta_2_cprestamos_personales
    , lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente
    , mcuenta_corriente - lag_1_mcuenta_corriente as delta_1_mcuenta_corriente
    , lag(mcuenta_corriente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_corriente
    , mcuenta_corriente - lag_2_mcuenta_corriente as delta_2_mcuenta_corriente
FROM competencia_01

,Success


In [51]:
%%sql
select 
    *
from competencia_01_lag_delta
limit 10

,numero_de_cliente,foto_mes,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter,delta_2_ctrx_quarter,lag_1_cpayroll_trx,delta_1_cpayroll_trx,lag_2_cpayroll_trx,delta_2_cpayroll_trx,...,lag_2_mtarjeta_visa_consumo,delta_2_mtarjeta_visa_consumo,lag_1_cprestamos_personales,delta_1_cprestamos_personales,lag_2_cprestamos_personales,delta_2_cprestamos_personales,lag_1_mcuenta_corriente,delta_1_mcuenta_corriente,lag_2_mcuenta_corriente,delta_2_mcuenta_corriente
0,249232117,202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249232117,202102,32.0,3.0,NaN,NaN,0.0,0.0,NaN,NaN,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
2,249232117,202103,35.0,0.0,32.0,3.0,0.0,3.0,0.0,3.0,...,41431.89,7606.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,249232117,202104,35.0,-1.0,35.0,-1.0,3.0,-2.0,0.0,1.0,...,48590.77,-3468.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,249232117,202105,34.0,-4.0,35.0,-5.0,1.0,0.0,3.0,-2.0,...,49038.13,-1188.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,249232117,202106,30.0,-1.0,34.0,-5.0,1.0,1.0,1.0,1.0,...,45122.76,5796.27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,249242331,202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,249242331,202102,95.0,-3.0,NaN,NaN,1.0,0.0,NaN,NaN,...,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN
8,249242331,202103,92.0,-2.0,95.0,-5.0,1.0,0.0,1.0,0.0,...,47103.57,-25257.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,249242331,202104,90.0,13.0,92.0,11.0,1.0,0.0,1.0,0.0,...,34811.27,-6636.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Ahora voy con el promedio de cada campo

In [52]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n    ,avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
FROM competencia_01
"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    , foto_mes 
    ,avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctrx_quarter
    ,avg(cpayroll_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll_trx
    ,avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll
    ,avg(mpayroll2) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll2
    ,avg(cpayroll2_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll2_trx
    ,avg(mpasivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpasivos_margen
    ,avg(mprestamos_personales) over (partition by numero_de_c

In [53]:
%%sql
CREATE OR REPLACE TABLE competencia_01_avg AS
select
    numero_de_cliente
    , foto_mes 
    ,avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_ctrx_quarter
    ,avg(cpayroll_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll_trx
    ,avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll
    ,avg(mpayroll2) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpayroll2
    ,avg(cpayroll2_trx) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cpayroll2_trx
    ,avg(mpasivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mpasivos_margen
    ,avg(mprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mprestamos_personales
    ,avg(mcuentas_saldo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuentas_saldo
    ,avg(mcaja_ahorro) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcaja_ahorro
    ,avg(mcomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcomisiones_mantenimiento
    ,avg(cproductos) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cproductos
    ,avg(cdescubierto_preacordado) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cdescubierto_preacordado
    ,avg(mtarjeta_visa_consumo) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mtarjeta_visa_consumo
    ,avg(cprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_cprestamos_personales
    ,avg(mcuenta_corriente) over (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row) as avg_3_mcuenta_corriente
FROM competencia_01

,Success


In [54]:
%%sql
select 
    *
from competencia_01_avg

,numero_de_cliente,foto_mes,avg_3_ctrx_quarter,avg_3_cpayroll_trx,avg_3_mpayroll,avg_3_mpayroll2,avg_3_cpayroll2_trx,avg_3_mpasivos_margen,avg_3_mprestamos_personales,avg_3_mcuentas_saldo,avg_3_mcaja_ahorro,avg_3_mcomisiones_mantenimiento,avg_3_cproductos,avg_3_cdescubierto_preacordado,avg_3_mtarjeta_visa_consumo,avg_3_cprestamos_personales,avg_3_mcuenta_corriente
0,249223005,202101,182.00,1.000000,115229.650000,0.0,0.0,728.450000,0.0,-1467.410000,43654.250000,0.0,6.0,1.0,9838.700000,0.0,-155.240000
1,249223005,202102,195.00,1.500000,123387.865000,0.0,0.0,648.060000,0.0,3725.495000,40710.425000,0.0,6.0,1.0,13851.170000,0.0,-436.235000
2,249223005,202103,197.00,1.333333,124111.216667,0.0,0.0,633.033333,0.0,228.066667,40288.313333,0.0,6.0,1.0,12904.783333,0.0,-469.613333
3,249223005,202104,196.25,1.500000,129189.817500,0.0,0.0,582.067500,0.0,1791.197500,40900.987500,0.0,6.0,1.0,14812.307500,0.0,-582.565000
4,249223005,202105,193.50,1.500000,137520.757500,0.0,0.0,626.842500,0.0,431.655000,46146.495000,0.0,6.0,1.0,14955.150000,0.0,-600.635000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,1594091333,202105,2.00,1.000000,47016.110000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,7.0,1.0,0.000000,0.0,0.000000
981942,1594091333,202106,25.50,1.500000,54997.260000,0.0,0.0,241.715000,0.0,81.100000,6501.465000,0.0,7.0,1.0,8879.480000,0.0,0.000000
981943,1594452692,202106,2.00,1.000000,58650.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,6.0,1.0,0.000000,0.0,0.000000
981944,1595194359,202106,38.00,2.000000,144796.750000,0.0,0.0,1540.090000,0.0,40466.650000,75023.260000,0.0,7.0,1.0,5180.690000,0.0,0.000000


## reg_slope

In [55]:
nuevos_features = ""
for campo in campos:
  nuevos_features +=  f"\n    ,regr_slope({campo}, cliente_antiguedad) over ventana_3 as {campo}_slope_3"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
    ,regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_3 as cpayroll_trx_slope_3
    ,regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as mpayroll_slope_3
    ,regr_slope(mpayroll2, cliente_antiguedad) over ventana_3 as mpayroll2_slope_3
    ,regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_3 as cpayroll2_trx_slope_3
    ,regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3
    ,regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as mprestamos_personales_slope_3
    ,regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as mcuentas_saldo_slope_3
    ,regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as mcaja_ahorro_slope_3
    ,regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as mco

In [56]:
%%sql
CREATE OR REPLACE TABLE competencia_01_regr_slope AS
select
    numero_de_cliente
    , foto_mes 
    ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
    ,regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_3 as cpayroll_trx_slope_3
    ,regr_slope(mpayroll, cliente_antiguedad) over ventana_3 as mpayroll_slope_3
    ,regr_slope(mpayroll2, cliente_antiguedad) over ventana_3 as mpayroll2_slope_3
    ,regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_3 as cpayroll2_trx_slope_3
    ,regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_3 as mpasivos_margen_slope_3
    ,regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_3 as mprestamos_personales_slope_3
    ,regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_3 as mcuentas_saldo_slope_3
    ,regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_3 as mcaja_ahorro_slope_3
    ,regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_3 as mcomisiones_mantenimiento_slope_3
    ,regr_slope(cproductos, cliente_antiguedad) over ventana_3 as cproductos_slope_3
    ,regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_3 as cdescubierto_preacordado_slope_3
    ,regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_3 as mtarjeta_visa_consumo_slope_3
    ,regr_slope(cprestamos_personales, cliente_antiguedad) over ventana_3 as cprestamos_personales_slope_3
    ,regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_3 as mcuenta_corriente_slope_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)

,Success


In [57]:
%%sql
select 
    *
from competencia_01_regr_slope

,numero_de_cliente,foto_mes,ctrx_quarter_slope_3,cpayroll_trx_slope_3,mpayroll_slope_3,mpayroll2_slope_3,cpayroll2_trx_slope_3,mpasivos_margen_slope_3,mprestamos_personales_slope_3,mcuentas_saldo_slope_3,mcaja_ahorro_slope_3,mcomisiones_mantenimiento_slope_3,cproductos_slope_3,cdescubierto_preacordado_slope_3,mtarjeta_visa_consumo_slope_3,cprestamos_personales_slope_3,mcuenta_corriente_slope_3
0,249328966,202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,249328966,202102,1.0,-1.0,31382.320,0.0,0.0,-722.810,0.0,-42174.860,-4696.750,0.0,0.0,0.0,13342.480,0.0,0.0
2,249328966,202103,4.5,-0.5,12318.755,0.0,0.0,-426.685,0.0,-22748.310,-2106.365,0.0,0.0,0.0,-2545.440,0.0,0.0
3,249328966,202104,3.2,-0.3,7550.850,0.0,0.0,-118.476,0.0,-9417.598,1.047,0.0,0.0,0.0,5269.316,0.0,0.0
4,249328966,202105,0.8,0.0,1252.336,0.0,0.0,-47.606,0.0,-1781.722,-3045.928,0.0,0.0,0.0,805.637,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981941,857495097,202101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
981942,857495097,202102,23.0,0.0,13296.760,0.0,0.0,-116.740,0.0,-5662.100,3698.370,0.0,0.0,0.0,10831.860,0.0,0.0
981943,857495097,202103,12.0,0.0,6648.380,0.0,0.0,-107.235,0.0,-1685.415,-2651.040,0.0,0.0,0.0,8435.630,0.0,0.0
981944,857495097,202104,9.1,0.0,3249.957,0.0,0.0,-83.408,0.0,1661.672,-5079.681,0.0,0.0,0.0,4988.086,0.0,0.0


In [58]:
nuevos_features = ""
for campo in campos:
  nuevos_features +=  f"\n    ,percent_rank() OVER (ORDER BY {campo}) as percent_rank_{campo}"


consulta_sql = f"""
CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    {nuevos_features.strip()}
from competencia_01

"""

print(consulta_sql)


CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    ,percent_rank() OVER (ORDER BY ctrx_quarter) as percent_rank_ctrx_quarter
    ,percent_rank() OVER (ORDER BY cpayroll_trx) as percent_rank_cpayroll_trx
    ,percent_rank() OVER (ORDER BY mpayroll) as percent_rank_mpayroll
    ,percent_rank() OVER (ORDER BY mpayroll2) as percent_rank_mpayroll2
    ,percent_rank() OVER (ORDER BY cpayroll2_trx) as percent_rank_cpayroll2_trx
    ,percent_rank() OVER (ORDER BY mpasivos_margen) as percent_rank_mpasivos_margen
    ,percent_rank() OVER (ORDER BY mprestamos_personales) as percent_rank_mprestamos_personales
    ,percent_rank() OVER (ORDER BY mcuentas_saldo) as percent_rank_mcuentas_saldo
    ,percent_rank() OVER (ORDER BY mcaja_ahorro) as percent_rank_mcaja_ahorro
    ,percent_rank() OVER (ORDER BY mcomisiones_mantenimiento) as percent_rank_mcomisiones_mantenimiento
    ,percent_rank() OVER (ORDER BY cproductos) as percent_rank_cproductos
   

In [59]:
%%sql
CREATE OR REPLACE TABLE competencia_01_percent_rank AS
select
    numero_de_cliente
    , foto_mes 
    ,percent_rank() OVER (ORDER BY ctrx_quarter) as percent_rank_ctrx_quarter
    ,percent_rank() OVER (ORDER BY cpayroll_trx) as percent_rank_cpayroll_trx
    ,percent_rank() OVER (ORDER BY mpayroll) as percent_rank_mpayroll
    ,percent_rank() OVER (ORDER BY mpayroll2) as percent_rank_mpayroll2
    ,percent_rank() OVER (ORDER BY cpayroll2_trx) as percent_rank_cpayroll2_trx
    ,percent_rank() OVER (ORDER BY mpasivos_margen) as percent_rank_mpasivos_margen
    ,percent_rank() OVER (ORDER BY mprestamos_personales) as percent_rank_mprestamos_personales
    ,percent_rank() OVER (ORDER BY mcuentas_saldo) as percent_rank_mcuentas_saldo
    ,percent_rank() OVER (ORDER BY mcaja_ahorro) as percent_rank_mcaja_ahorro
    ,percent_rank() OVER (ORDER BY mcomisiones_mantenimiento) as percent_rank_mcomisiones_mantenimiento
    ,percent_rank() OVER (ORDER BY cproductos) as percent_rank_cproductos
    ,percent_rank() OVER (ORDER BY cdescubierto_preacordado) as percent_rank_cdescubierto_preacordado
    ,percent_rank() OVER (ORDER BY mtarjeta_visa_consumo) as percent_rank_mtarjeta_visa_consumo
    ,percent_rank() OVER (ORDER BY cprestamos_personales) as percent_rank_cprestamos_personales
    ,percent_rank() OVER (ORDER BY mcuenta_corriente) as percent_rank_mcuenta_corriente
from competencia_01

,Success


In [ ]:
## cree 5 tablas
## las sumas que me parecian mas unos ratios..
## otro con lag y delta de dos meses
## promedio de todos los campos
## regr_slope de todos los campos
## per_Rank

##.. ahora tengo que unir esto

In [4]:
%%sql
SELECT * FROM competencia_01_percent_rank 


,numero_de_cliente,foto_mes,percent_rank_mcuentas_saldo,percent_rank_cpayroll_trx,percent_rank_ctrx_quarter,percent_rank_mcaja_ahorro,percent_rank_mprestamos_personales,percent_rank_mtarjeta_visa_consumo,percent_rank_mpasivos_margen,percent_rank_mcomisiones_mantenimiento,percent_rank_cproductos
0,600226432,202102,0.054622,0.000000,0.000000,0.000069,0.000000,0.000000,0.032022,0.728090,0.000000
1,600226432,202101,0.064648,0.000000,0.000000,0.000069,0.000000,0.000000,0.032022,0.728090,0.000000
2,1168447427,202101,0.161096,0.000000,0.049220,0.106370,0.915574,0.000000,0.058809,0.728090,0.000000
3,1168447427,202102,0.157554,0.000000,0.063402,0.149362,0.914701,0.000000,0.076374,0.728090,0.000000
4,985836090,202101,0.396031,0.000000,0.067243,0.404743,0.892866,0.000000,0.242165,0.728090,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
981941,747703842,202103,0.954645,0.459453,0.971773,0.558679,0.866152,0.813846,0.013990,0.014346,0.999952
981942,747703842,202102,0.874922,0.459453,0.971449,0.594720,0.868405,0.915984,0.029428,0.014346,0.999952
981943,316695411,202102,0.550156,0.750350,0.991801,0.471971,0.900591,0.992488,0.151916,0.014346,0.999998
981944,316695411,202104,0.619428,0.459453,0.992751,0.410720,0.845084,0.983428,0.838171,0.014346,0.999998


In [3]:
%%sql 
create or replace table competencia_01_fe as
SELECT * FROM competencia_01 as base

LEFT JOIN competencia_01_suma_division as suma
ON base.numero_de_cliente = suma.numero_de_cliente 
    AND base.foto_mes = suma.foto_mes  

LEFT JOIN competencia_01_lag_delta as lag
ON suma.numero_de_cliente = lag.numero_de_cliente 
    AND suma.foto_mes = lag.foto_mes  

LEFT JOIN competencia_01_avg as avg 
ON suma.numero_de_cliente = avg.numero_de_cliente 
    AND suma.foto_mes = avg.foto_mes  
    
LEFT JOIN competencia_01_regr_slope as reg 
ON suma.numero_de_cliente = reg.numero_de_cliente 
    AND suma.foto_mes = reg.foto_mes

LEFT JOIN competencia_01_percent_rank as p_rank
ON suma.numero_de_cliente = p_rank.numero_de_cliente 
    AND suma.foto_mes = p_rank.foto_mes

,Success


In [4]:
%%sql
COPY competencia_01_fe TO '../../../datasets/competencia_01_fe.csv' (FORMAT CSV, HEADER TRUE)

,Success


In [12]:
%%sql
select * from competencia_01_fe 
limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,ratio_seguros_vs_consumo,fechamora_mayor,fechamora_menor,fechacierre_mayor,fechacierre_menor,fvencimiento_mayor,fvencimiento_menor,fechaalta_mayor,fechalta_menor,cliente_antiguedad_6
0,597882044,202104,1,0,0,38,34,-627.55,-24128.23,734.30,...,0.000096,NaN,NaN,2,2,-1399,-1764,449,449,2
1,597882044,202105,1,0,0,38,35,-566.64,-25269.15,398.82,...,NaN,NaN,NaN,5,5,-1368,-1733,480,480,2
2,597882044,202106,1,0,0,38,36,-1544.20,-26048.79,345.46,...,NaN,NaN,NaN,0,0,-1338,-1703,510,510,2
3,598265274,202103,1,0,0,59,197,3474.62,23325.33,308.41,...,0.000000,NaN,NaN,7,7,-1674,-1674,4557,4557,8
4,598265274,202104,1,0,0,59,198,3270.27,25403.55,439.82,...,NaN,NaN,NaN,9,9,-1644,-1644,4587,4587,8
5,598265274,202105,1,0,0,59,199,3059.34,27463.42,267.03,...,NaN,NaN,NaN,12,12,-1613,-1613,4618,4618,8
6,598272063,202101,1,0,0,36,195,59.75,8516.51,332.95,...,0.000000,NaN,NaN,4,4,-576,-1154,5445,4904,8
7,598272063,202102,1,0,0,36,196,-2657.38,5384.58,3431.49,...,0.000000,NaN,NaN,-3,-3,-548,-1126,5473,4932,8
8,598272063,202103,1,0,0,36,197,-8602.96,-5381.94,155.66,...,0.000000,NaN,NaN,1,1,-517,-1095,5504,4963,8
9,598272063,202105,1,0,0,36,199,-5557.74,-20470.53,253.59,...,0.000000,NaN,NaN,5,5,-456,-1034,5565,5024,8
